In [1]:
!pip install -q kaggle
!gdown "1CiOTLdUpSZ3d-2io-tALvhoaqxDVMXw2"

Downloading...
From: https://drive.google.com/uc?id=1CiOTLdUpSZ3d-2io-tALvhoaqxDVMXw2
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 369kB/s]


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d "mohamedmustafa/real-life-violence-situations-dataset"

100% 3.58G/3.58G [03:16<00:00, 14.7MB/s]
100% 3.58G/3.58G [03:16<00:00, 19.6MB/s]


In [4]:
!unzip /content/real-life-violence-situations-dataset.zip > extract.txt

In [5]:
pip install pytorchvideo torchvideo pytorch_lightning > install.txt

In [6]:
import torchvideo
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split

In [7]:
# !mv "Real Life Violence Dataset" Real_Life_Violence_Dataset

In [8]:
os.listdir("Real Life Violence Dataset")

['NonViolence', 'Violence']

In [9]:
# non = glob("Real Life Violence Dataset/NonViolence/*")
# violence = glob("Real Life Violence Dataset/Violence/*")

# labels = [0] * len(non) + [1] * len(violence)

In [10]:
# data = pd.DataFrame(zip(non + violence , labels), columns=['file', 'label'])
# data['file'] = "/content/" + data['file']
# data.head()

In [11]:
# train_df , test_df = train_test_split(data , test_size = .2 , shuffle =True)
# train_df.shape , test_df.shape

In [12]:
# ! mkdir data
# train_df.to_csv('data/my_file.csv')

In [13]:
from pytorchvideo.data import LabeledVideoDataset , make_clip_sampler , labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

from torch.utils.data import DataLoader

import pytorchvideo

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [14]:
video_trans = Compose([
    ApplyTransformToKey(key='video',
    transform =Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x / 255),
        Normalize((0.45, 0.45 , 0.45),(0.225, 0.225,0.225)),
        RandomShortSideScale(min_size=248 , max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(0.5)

    ]))
])

In [15]:
# train_dataset = labeled_video_dataset("Real Life Violence Dataset" , clip_sampler =make_clip_sampler('random',2),
#                                      transform=video_trans,decode_audio=False
#                                      )

# train_loader = DataLoader(train_dataset, batch_size =4 , num_workers= 2 , pin_memory=True)

In [16]:
# batch = next(iter(train_loader))
# batch.keys()

In [17]:
# batch['label'].shape

In [18]:
import torch.nn as nn
import torch
from pytorch_lightning import LightningModule ,seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint , LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchmetrics
from sklearn.metrics import classification_report

In [19]:
# base_model = torch.hub.load("facebookresearch/pytorchvideo",model = 'efficient_x3d_xs' , pretrained=True)
# # base_model

In [20]:
class CustomModel(LightningModule):

  def __init__(self):
    super().__init__()
    self.base_model = torch.hub.load("facebookresearch/pytorchvideo",model = 'efficient_x3d_xs' , pretrained=True)

    self.linear = nn.Linear(400 , 1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()


    self.batch_size = 2
    self.num_worker = 0
    self.lr = 1e-3

    self.loss_fn = nn.BCEWithLogitsLoss()
    self.metric = torchmetrics.classification.BinaryAccuracy()

    self.training_step_outputs = []


  def forward(self, video):

    net =self.base_model(video)

    net = self.relu(net)

    net = self.linear(net)

    net = self.sigmoid(net)

    return net

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters() , lr= self.lr)
    scheduler = CosineAnnealingLR(optimizer , T_max=10 , eta_min = 1e-6 , last_epoch=-1)

    return {"optimizer" : optimizer , "scheduler" : scheduler}

  def train_dataloader(self):
    train_dataset = labeled_video_dataset("Real Life Violence Dataset" , clip_sampler =make_clip_sampler('random',2),
                                     transform=video_trans,decode_audio=False
                                     )

    train_loader = DataLoader(train_dataset, batch_size =self.batch_size , num_workers= self.num_worker , pin_memory=True)

    return train_loader



  def training_step(self, batch, batch_idx):

    video, label = batch['video'] , batch['label']

    output = self(video).squeeze()

    output = torch.round(output)



    # print(f"tpye of ouput is {output.dtype} , type of label is {label.dtype}")

    # print(output , label)

    loss = self.loss_fn(output , label.float())


    metric = self.metric(output , label.to(torch.int64))

    self.training_step_outputs.append( {"loss": loss , "metric" : metric.detach() })

    return loss


  def on_train_epoch_end(self):

    loss = torch.stack([x['loss'] for x in self.training_step_outputs]).mean().cpu().numpy().round(2)
    metric = torch.stack([x['metric'] for x in self.training_step_outputs]).mean().cpu().numpy().round(2)

    self.log("training Accuracy ", metric)
    self.log("Training Loss" , loss)

    self.training_step_outputs.clear()





In [21]:
model = CustomModel()
# output = model.training_step(batch,0)

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/efficient_x3d_xs_original_form.pyth" to /root/.cache/torch/hub/checkpoints/efficient_x3d_xs_original_form.pyth
100%|██████████| 14.8M/14.8M [00:00<00:00, 230MB/s]


In [22]:
seed_everything(0)
trainer = Trainer(max_epochs=1,accelerator='gpu',
                  precision=16,accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0)

INFO:lightning_fabric.utilities.seed:Global seed set to 0
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:361: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | base_model | EfficientX3d      | 3.8 M 
1 | linear     | Linear            | 401   
2 | relu       | ReLU              | 0     
3 | sigmoid    | Sigmoid           | 0     
4 | loss_fn    | BCEWithLogitsLoss | 0     
5 | metric     | BinaryAccuracy    | 0     
-------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.179    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]